# Bike Sharing Demand

- 자전거 수요 예측 노트북 
- 모델링 보다는 변수들의 관계를 알아가보는데 초점을 두었습니다.


### 변수 설명
- datetime: 시간 
- season : 계절
- holiday : 공휴일
- workingday : 일하는 날 
- weather : 날씨(맑은날이 1, 가장 안좋은 날이 4)
- temp    : 온도
- atemp   : 체감 온도
- humidity : 습도
- windspeed : 풍속
- count   : 수요량(target)
## 목차
 1. [데이터 로딩 및 확인](#loading) 
 
    1.1  데이터 로딩
    
    1.2  train, test 정보 확인 
    
    1.3  결측치 확인 
    
    1.4  feature의 타입 확인 
     
 2. [전처리](#전처리)
    
    2.1 feature 타입 변환
    
    2.2 feature 변환
    
 3. [EDA](#EDA) 
 
    3.1 target 분포 확인
    
    3.2 feature 분포 확인
    
    3.3 feature간 관계
    
    - pairplot
    
    - skewness
     
    - heatmap  	
     
    3.4 target feature 관계
      
    - 산점도(수치형)
       
    - boxplot(범주형)
  
    	
4. [feature engineering](#feature)

   4.1 one-hot encoding
   
   4.2 randomforest를 이용한 feature importances 확인 	

5. [modeling](#modeling)

   5.1 회귀 머신러닝으로 학습
   
   5.2 모델 선택 및 튜닝
   
   5.3 오버피팅 확인 

6. [submission](#submission)

## 1. 데이터 로딩 및 확인
<a id='loading'></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.pipeline import Pipeline
from scipy.stats import norm, skew, kurtosis
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from xgboost import XGBRegressor

import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

### 1.1 데이터 로딩 

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
train=pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
sub=pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

### 1.2 데이터 정보 확인 

In [ ]:
print(train.shape)
print(train.columns)

In [ ]:
print(test.shape)
print(test.columns)

- test set에는 casual, registered가 없기 때문에 삭제 하겠다.

In [ ]:
del train['casual']
del train['registered']

### 1.3 결측치 확인

In [ ]:
print('=====train=====')
print(train.isnull().sum())
print('=====test=====')
print(test.isnull().sum())

- 결측치가 없다.

### 1.4 feature의 타입 확인 

In [ ]:
train.info()

- datetime이 object 타입으로 되어있다. 
- season, holiday, workingday, weather 변수는 범주형 변수인데 int 타입으로 되어있다. 

## 2. 전처리 
<a id='전처리'></a>

### 2.1 feature 타입 변환
- 위에서 확인한 범주형 변수를 category로 지정해준다. 

In [ ]:
for val in ['season','holiday','workingday','weather']:
    train[val] = train[val].astype('category')
    test[val] = test[val].astype('category')

### 2.2 feature 변환

- datetime을 연,월,일,요일,시간으로 분리해 새로운 feature로 만들어 준다. 

In [ ]:
def datetime_split(df):
    df['datetime']=df.datetime.apply(pd.to_datetime)
    df['year']=df.datetime.apply(lambda x : x.year)
    df['dayofweek'] = df.datetime.apply(lambda x : x.dayofweek)
    df['month']=df.datetime.apply(lambda x : x.month)
    df['day']=df.datetime.apply(lambda x : x.day)
    df['hour']=df.datetime.apply(lambda x  : x.hour)
    
datetime_split(train)
datetime_split(test)

## 3. EDA
<a id = 'EDA'></a>

### 3.1 target 분포

In [ ]:
target = train['count']
sns.distplot(target, fit = norm)

plt.annotate('skewness: {0}'.format(np.round(skew(target),3)), xy = (20,0.006))
plt.annotate('kurtosisness: {0}'.format(np.round(kurtosis(target),3)), xy = (20,0.0065))

- 종속변수인 count의 분포를 보았을때 왼쪽으로 치우친 모습을 보인다.
- 정규화를 통해 분포의 치우침을 완화 해볼 필요가 있어 보인다. 

In [ ]:
def scaler(x) :
    return (x - np.mean(x)) / np.std(x)

sns.distplot(scaler(target))
print('mean :',np.mean(scaler(target)))
print('std  :',np.std(scaler(target)))
plt.annotate('skewness: {0}'.format(np.round(skew(scaler(target)),3)), xy = (2,1.05))
plt.annotate('kurtosisness: {0}'.format(np.round(kurtosis(scaler(target)),3)), xy = (2,1))

- 종속 변수가 연속형 변수가 아니다보니 정규화(normalization)를 통해서는 분포가 변하지 않으므로 log-scale을 해줄 필요가 있어보인다.

In [ ]:
sns.distplot(np.log1p(target), fit = norm)

plt.annotate('skewness: {0}'.format(np.round(skew(np.log1p(target)),3)), xy = (1,0.35))
plt.annotate('kurtosisness: {0}'.format(np.round(kurtosis(np.log1p(target)),3)), xy = (1,0.32))

- log를 취한 결과 분포의 왜도와 첨도가 조금 더 정규분포에 근사한 모습을 보인다. 
- count에 log를 취하여 예측해 보려 한다. 다만 0으로 값을 가지는 데이터가 많으므로 log 대신 log1p를 취했다. 

In [ ]:
train['count']  = np.log1p(target)

### 3.2 feature 분포 확인 

### pairplot
- feature의 분포와 다른 feature간의 관계를 한번에 보여주는 pairplot을 통해 확인해보겠다

In [ ]:
sns.pairplot(train[['temp', 'atemp', 'humidity', 'windspeed']])

- temp 와 atemp간의 뚜렷한 선형관계가 보인다. 다른 feature들간의 관계는 독립적인 모습을 보인다.ㅅ
- feature간의 강한 선형관계가 있을 경우 회귀에서 추정계수의 분산이 높아지므로 신뢰할 수 없게 된다. 이를 다중공선성 현상이라 하는데 해결방법으로는 변수 제거, 차분, log 변환, 정규화 등이 있다.
- 온도와 체감온도는 비슷한 의미를 가지기 때문에 변환을 통해서 다중공선성 현상을 해결하기 보다는 둘 중 하나를 삭제 하는것이 좋아보인다.
- target과 더 낮은 상관계수를 가지는 변수를 제거하도록 하겠다. 


In [ ]:
train[['temp','atemp','count']].corr(method = 'pearson')

- 큰 차이는 없지만 temp가 target과 조금 더 높은 상관계수를 가지기에 atemp를 제거 하도록 한다.

In [ ]:
del train['atemp']
del test['atemp']

### skewness(왜도)

- feature의 분포는 pairplot을 통해 시각화하여 확인한 결과 대부분 정규분포의 모습을 보이지만 왜도를 직접 확인해보겠다.
- 왜도는 좌우 대칭에 대한 통계요약치다. 분포의 3차적률과 관련이 있으며 0에 가까울 수록 정규분포에 근사한다고 말할수 있다. 
- 수치형 변수들의 왜도를 확인해보겠다.

In [ ]:
for val in ['windspeed','humidity','temp']:
    print('{}` skewness : {:.3f}'.format(val, skew(train[val])))

- 왜도의 정규성 기준은 절댓값 2로 알려져있으나 조금 더 엄격하게 기준을 설정 한다면 0.75정도로 설정한다.
- 수치형 feature들의 왜도는 절대값이 0에 가까운 값을 가지기에 정규분포에 근사하다고 말할 수 있다. 

### 3.4 feature와 target의 관계 
- 수치형 변수는 산점도(scatter)를 통해, 범주형 변수는 boxplot을 통해 관계를 알아가보려 한다.



#### Target ~ Year

In [ ]:
sns.boxplot(x = train['year'],
            y = train['count'])

- box의 크기의 차이는 없어보이나 2012년일 때 평균이 더 높다. 또한 2012년일때 이상치가 더 많이 보이는것을 확인할수 있다. 

In [ ]:
sns.boxplot(x = train['season'],
            y = train['count'])

- season별 count 의 boxplot 모양의 차이는 크게 보이지 않는다. 다만 season1(봄)일때 다른 season에 비해 다른 위치에 있는것을 확인했다.
- season 3일때 다른 season에 비해 이상치가 많아 보인다.

#### Target ~ Hour

In [ ]:
sns.boxplot(x = train['hour'],
            y = train['count'])

- 새벽 시간에는 줄어 들다가 새벽 5시 부터 다시 증가 하는 모습을 보인다. 새벽 시간의 boxplot은 box 크기가 다른 시간대에 비해 크고 꼬리 또한 길다. 
- 아침 9시 부터 box의 크기가 줄어들지만 이상치가 생기는 것을 확인 할 수 있다. 아침 8시 부터 23시 까지 box의 모양은 크게 다르지 않는 모습을 보인다.
- 17시 부터 점차 적으로 count가 줄어드는 것을 확인 할 수 있다. 
- 시간(hour)과 count는 선형적 관계를 보이지 않는다. 시간을 numeric 변수로 사용하기엔 부적절해 보인다.
- 모든 시간대를 범주형 변수로 볼 경우 너무 많은 level을 가지게 된다. one-hot encoding을 하게 된다면 총 24개의 컬럼이 추가적으로 생기게 된다. 그러기엔 데이터의 양이 부족해보인다.
- 시간은 돌고 돌기 때문에 시간의 시작이 0시 일 필요는 없어 보인다. 
- 0~4시를 23시 이후의 시간대로 생각을 해보고 시간 중 제일 이른 시간을 5시로 생각해 보았다. 
- 그러기 위해서는 0~4시를 그대로 놔두기 보다는 0~4시를 24~28시로 바꾼 뒤 시간의 크기를 살린채 수치형 변수로 사용해 보았다.

In [ ]:
for i in [0,1,2,3,4] :
    train['hour'].replace(i, i+24, inplace = True)
    test['hour'].replace(i, i+24, inplace = True)
    
    
train.hour.value_counts()

In [ ]:
plt.scatter(x = train['hour'],
            y = train['count'],
           alpha = 0.3,
           color = 'blue')

plt.xlabel('hour')
plt.ylabel('count')

plt.title('hour ~ count')

sns.regplot(x = train['hour'],
            y = train['count'],    # regplot(degree = 2)
           order = 2, label = 'degree.2')

sns.regplot(x = train['hour'],
           y = train['count'],     # regplot(degree = 1)
           order = 1, label = 'degree.1')

plt.legend(loc = 'upper right')
plt.show()

- 선형적 관계는 보이지 않지만 뚜렷한 비선형 관계를 보인다. 
- hour를 다항식으로 변환해볼 필요가 있다고 생각된다.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
# linear model 
lr_m = LinearRegression()
poly_m = LinearRegression()

train_hour = train.loc[:, 'hour'].values
test_hour = test.loc[:, 'hour'].values

train_hour = train_hour.reshape(-1, 1)
test_hour = test_hour.reshape(-1, 1)

poly = PolynomialFeatures(degree=2)

train_hour_sqr = poly.fit_transform(train_hour)
test_hour_sqr = poly.fit_transform(test_hour)

lr_m.fit(train_hour, target)
poly_m.fit(train_hour_sqr, target)

pred_lr = lr_m.predict(train_hour)
pred_poly = poly_m.predict(train_hour_sqr)

lr_score = mean_squared_error(target, pred_lr)
poly_score = mean_squared_error(target, pred_poly)

r2_lr = r2_score(target, pred_lr)
r2_poly = r2_score(target, pred_poly)

print('r2_score (degree = 1) : {0:.3f} \n  MSE : {1:.3f}'.format(r2_lr, lr_score))
print('======================================================')
print('r2_score(degree = 2) : {0:.3f} \n MSE : {1:.3f}'.format(r2_poly, poly_score))
print('======================================================')
print('polynomial regression estimators : ({1:.3f}) * hour^2 + ({0:.3f}) * hour + ({2:.3f})'.format(poly_m.coef_[1],poly_m.coef_[2], poly_m.intercept_))

- hour를 2차항으로 변환했을 떄 r2 score와 mse 지표가 훨씬 더 좋게 나타난다.
- hour의 2차항의 추정계수를 살펴보면 1차항은 양수, 2차항은 음수로 나타난다. 이는  hour가 증가할때 count가 증가하지만 증가량이 한계체감한다는 뜻으로 해석 할 수 있다. 

In [ ]:
train['hour_sqr'] = np.square(train['hour'])
test['hour_sqr'] = np.square(test['hour'])

#### Target ~ Holiday : Hour

In [ ]:
train.holiday.value_counts()

- holiday에는 주말이 포함이 되지 않았다.
- 개인적인 생각으로는 쉬는날과 평일에 시간별 자전거 수요가 다를것이라 생각한다.
- 따라서 토요일, 일요일일때도(dayofweek = 5,6) holiday를 1로 변환해주겠다.

In [ ]:
train.loc[train.dayofweek >= 5,'holiday'] = 1
test.loc[test.dayofweek >= 5, 'holiday'] = 1

In [ ]:
train.holiday.value_counts()

In [ ]:
sns.boxplot(x = train['holiday'],
            y = train['count'])

- holiday별 count의 분포는 차이가 많이 없어 보이므로 설명변수로 유의미하지 않을 것 같다. 
- 다만 휴일일때와 아닐때 시간별로 자전거 대여 수요량이 다를것으로 기대해볼수 있다. 
- 휴일 낮과 오후사이에 수요량이 평일보다 더 많을것으로 예상되고 출퇴근 시간에는 수요량이 평일보다 더 적을것으로 예상된다.

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x = train['hour'],
            y = train['count'],
            hue = train['holiday'])

- 휴일일때와 아닐때의 시간별 target의 분포가 확실히 다른것을 확인할수 있다.
- 출퇴근시간(5~9 , 18~밤)에는 평일에 더 많이 빌리고 출퇴근시간이 아닐때는 휴일에 더 많이 빌린다고 말할수 있다. 
- holiday와 count만의 boxplot을 그려보았을때는 큰 차이가 없어보였지만 이는 시간대별로 차이가 있기때문이다. 
- holiday와 hour의 상호작용변수를 추가적으로 만들어줄 필요가 있어 보인다.
- 휴일 일때 5시부터 9시 까지는 평일보다 적게 빌리지만 10시부터 17시까지는 휴일에 더 많이 빌린다. 또한 18시부터 23시까지 적게 빌리다가 새벽에는 더 많이 빌리는 형태를 보인다. 

### Target ~ Month

In [ ]:
sns.boxplot(x = train['month'],
            y = train['count'])

In [ ]:
plt.scatter(x = train['month'],
            y = train['count'])

- month 변수를 numeric 변수로 생각할 경우 뚜렷한 관계나 추세를 보이지 않는다. 
- 5~8월에는 다른 달에 비해 box의 크기가 작고 꼬리가 긴 모습을 보인다. 

In [ ]:
plt.scatter(x = train['windspeed'],
            y = train['count'])
plt.xlabel('windspeed')
plt.ylabel('count')

- windspeed와 count의 산점도를 그려 보았을 때 이상한 점이 windspeed가 0을 가지는 값이 많다.
- 풍속이 0이라는게 현실에서 불가능하지는 않지만 0과 6에 공백이 생기는 것을 확인했다.
- kaggle Discussion 에서는 0~6 사이의 풍속을 가지면 풍속이 너무 낮아 측정이 안됐을것이라고 말한다. 

In [ ]:
sns.boxplot(x = train['weather'],
            y = train['count'])

In [ ]:
print('train \n',train.weather.value_counts())
print('test \n',test.weather.value_counts())

- weather 를 4로 값을 가지는 데이터는 train set에서는 1개, test set에서는 2개다. weather가 4인 데이터가 너무 적기때문에 변수를 변환할 필요를 느꼈다.
- 데이터 설명에 보면 값이 커질수록 날씨가 안좋아 진다는 것을 알 수 있다. 
- weather 3 부터 비나 눈 혹은 태풍이 오는 날씨를 뜻한다.
- 따라서 4로 값을 가지는 데이터를 3으로 바꿔주도록 하겠다.

In [ ]:
train['weather'].replace(3,4,inplace = True)
test['weather'].replace(3,4,inplace = True)

In [ ]:
plt.scatter(x = train['humidity'],
            y = train['count'],
           alpha = 0.5,
           color = 'b')
plt.xlabel('humidity')
plt.ylabel('count')

plt.annotate('correlation between humidity and count : {0:.3f}'.format(train[['humidity','count']].corr().iloc[1,0]), xy = (0,6))
sns.regplot(x = train['humidity'],
            y = train['count'])

- count와 산점도를 그려 보았을때 상관계수는 -0.333으로 약한 선형적 관계를 보인다.

In [ ]:
sns.boxplot(x = train['dayofweek'],
            y = train['count'])

- 요일별 count의 분포 차이는 없어 보인다. 
- 휴일과 마찬가지로 시간과 연관이 있을것으로 예상이 된다. 

In [ ]:
plt.scatter(x = train['hour'],
            y = train['count'],
            c = train['dayofweek'])

- 요일별 target의 분포가 시간대 별로 다르다.
- 요일별 target의 boxplot만 보았을때는 차이가 없어보이지만 시간과 연관지어 보면 차이가 보인다. 

In [ ]:
plt.scatter(x = train['temp'],
            y = train['count'])

plt.annotate('correlation between humidity and count : {0:.3f}'.format(train[['temp','count']].corr().iloc[1,0]), xy = (0,6))
sns.regplot(x = train['temp'],
            y = train['count'])

-  temp가 증가할수록 count가 증가하는 상관관계를 보인다. 

In [ ]:
sns.heatmap(train[['temp','windspeed','humidity','hour','hour_sqr','count']].corr(method = 'pearson'),annot = True, fmt = '.2f')

- 피어슨 상관계수로 heatmap을 그려보았을 때 연속형 변수들 간의 상관계수는 대체적으로 낮은편이다.

- 다중공선성의 문제는 발생하지 않을것으로 예상된다. 

<a id = 'feature'></a>
    
## 4. feature engineering 

- modeling에 필요한 형태로 변수를 변환

### 4.1 randomforest를 이용한 feature importances

- 변수 중요도(feature importances)를 출력해주는 모델로는 lasso, xgboost, randomforest등이 있다.
- 여기서 randomforest를 사용하는 이유는 eda단계에서 feature들이 개별로 target에 영향을 주기 보다는 다른 feature와 연관되어 target에 영향을 줄 것으로 예상하기 때문이다.
- 예를 들어 휴일(holiday)같은 경우엔 혼자서 target과의 관계는 별차이 없어보이나 시간과 연관을 지었을때 뚜렷한 차이를 보였다. 
- 하지만 차이가 일정하게 나타나지는 않았다. 따라서 여러개의 의사결정나무를 생성하여 예측하는 랜덤 포레스트가 이 부분을 잘 설명해줄수 있으리라 믿기때문에 randomforest를 사용해 변수 중요도를 출력해보겠다.

In [ ]:
train.info()

- datetime은 모델이 학습할수 없으므로 삭제하겠다.

In [ ]:
del train['datetime']
del test['datetime']

### 모델 생성


#### 평가 지표 생성(root mean square lesat error)

In [ ]:
def rmsle(y, pred):
    log_y=np.log1p(y+1)
    log_pred=np.log1p(pred + 1)
    squared_error=(log_y-log_pred)**2
    rmsle=np.sqrt(np.mean(squared_error))
    return rmsle

rmsle_score = make_scorer(rmsle)

#### train_test_분리 및 randomforest 학습

In [ ]:
target = train['count']
train.drop('count', axis = 1, inplace = True) # target, feature 분리

In [ ]:
rf_reg = RandomForestRegressor(n_estimators=1000, n_jobs = -1, random_state = 777) # 1000개의 의사결정 나무 생성, cpu 집중, 난수 고정

x_train, x_test, y_train, y_test = train_test_split(train, target, test_size = 0.3, random_state = 777)

rf_reg.fit(x_train, y_train)

pred = rf_reg.predict(x_test)

pred = np.expm1(pred)
y_test = np.expm1(y_test)

print('RandomForest score : ', rmsle(pred, y_test))

In [ ]:
feat_imp = {'col' : train.columns,
            'importances' : rf_reg.feature_importances_}

feat_imp = pd.DataFrame(feat_imp).sort_values(by = 'importances', ascending = False)

sns.barplot(x = feat_imp['col'] ,
            y = feat_imp['importances'])
plt.xticks(rotation =  60)


In [ ]:
print(feat_imp)

In [ ]:
pred = rf_reg.predict(test)
pred = np.expm1(pred)
sub['count'] = pred
sub.to_csv('submission_20200202.csv',index = False)

- 모든 feature를 이용해 예측한 후 제출한 결과 약 0.44의 점수를 얻었다.
- 변수 중요도를 보면 weather, day, windspeed, holiday는 중요도가 0.01 보다 낮다
- 이 feature들을 제거한 후 모델링을 하는것이 좋아보인다.

In [ ]:
for val in ['weather','day','windspeed','holiday']:
    del train[val]
    del test[val]

## 4.2 one-hot encoding

In [ ]:
train_m = pd.get_dummies(train, columns=['month','year','dayofweek','workingday','season'])
test_m = pd.get_dummies(test,columns=['month','year','dayofweek','workingday','season'])
print(train_m.shape)
print(test_m.shape)

<a id='loading'></a>
## 5. Modeling


### 5.1 회귀 머신러닝으로 학습
여러 회귀 모델들의 예측성능을 확인해본 뒤 제일 좋은 모델을 선택하겠다.  
> 1, Randomforest  
> 2. XGBboost  
> 3. Linear Regression  
> 4. Regularizaed Linear Regression(Ridge, Lasso, ElasticNet)

In [ ]:
rf_reg=RandomForestRegressor()
xgb_reg=XGBRegressor()
lr_reg=LinearRegression()
lasso=Lasso()
ridge=Ridge()
elastic=ElasticNet()

model_list=[rf_reg, xgb_reg, lr_reg, ridge, lasso , elastic]
for model in model_list :
    score = cross_val_score(model, train_m, target , scoring = rmsle_score, cv = 3)
    print('{0} ` score : {1}'.format(model.__class__.__name__, np.mean(score)))

- target에 로그를 씌운 뒤 또 다시 로그를 씌운 점수이기 때문에 실제 점수와는 다른 단위를 가진다. 
- 이중에서 randomforest 로 예측을 한 뒤 제출 해 보겠다.

### 5.2 model tuning

In [ ]:
params = {'max_depth': [3,5,7,11],
         'min_samples_split': [2,4,6,8],
         "min_weight_fraction_leaf": [0.01,0.1,0.2,0.3],
         "max_features":[4,5,6]}
grid_rf = GridSearchCV(rf_reg,param_grid = params, n_jobs=-1)
grid_rf.fit(train_m, target)


In [ ]:
print(grid_rf.best_params_)

In [ ]:
pred = grid_rf.predict(test_m)

sub['count'] = np.expm1(pred)

sub.to_csv('randomForest.csv', index = False)